# jmespath lib

A biblioteca **`jmespath`** em Python é uma implementação da linguagem de consulta **JMESPath (JSON MEssaging Specification Path)**. JMESPath é uma linguagem de consulta declarativa projetada para extrair e transformar elementos de um documento JSON (ou estruturas de dados Python equivalentes).

Enquanto o JSONPath (como implementado por `jsonpath-ng`) é primariamente focado em *extração*, o **JMESPath vai além**, oferecendo poderosas capacidades de **transformação e remodelação** dos dados, além de funções integradas para processamento de valores. Isso o torna incrivelmente útil para:

  - Extrair subconjuntos de dados.
  - Remodelar a estrutura de documentos JSON.
  - Filtrar elementos com base em condições complexas.
  - Mapear dados para diferentes formatos.

In [1]:
%pip install jmespath

Note: you may need to restart the kernel to use updated packages.


## Como `jmespath` Funciona?

O processo de uso de `jmespath` é bastante direto:

1.  Você define uma **expressão JMESPath** como uma string.
2.  Você usa a função principal `jmespath.search()` para aplicar essa expressão a um documento JSON (ou um dicionário/lista Python).
3.  A função retorna o resultado da consulta, que pode ser um valor único, uma lista, um dicionário, ou `None` se nada for encontrado.

## Métodos e Propriedades Principais

A biblioteca `jmespath` é mais simples em sua API de alto nível do que `jsonpath-ng` ou `lxml`, focando-se principalmente em uma função de busca central.

Vamos usar o mesmo documento JSON de exemplo para demonstrações:

In [2]:
import jmespath
import json
from data import json_data # data.py

In [3]:
def run_jmespath_query(query: str):
    """
    Executes a JMESPath query and prints the results.
    """
    print(f"\n--- JMESPath Query: '{query}' ---")
    try:
        result = jmespath.search(query, json_data)
        if result is not None and result != []:
            print(f"  Resultado: {result}")
        else:
            print("  Nenhum resultado encontrado ou resultado vazio.")
    except Exception as e:
        print(f"  Erro ao executar query: {e}")

### 1. Função Principal: `jmespath.search()`

  - **O que faz:** É a função central para executar uma expressão JMESPath.
  - **Sintaxe:** `jmespath.search(expression, data)`
      - `expression`: A string JMESPath que você deseja executar.
      - `data`: O documento JSON (como um dicionário ou lista Python) no qual a pesquisa será realizada.
  - **Retorna:** O resultado da avaliação da expressão, que pode ser um tipo Python (`dict`, `list`, `str`, `int`, `float`, `bool`, `None`) correspondente aos tipos JSON.


In [4]:
# Selecionando um valor simples
run_jmespath_query('livraria.titulo')

# Selecionando o título do primeiro livro (índice 0)
run_jmespath_query('livraria.livros[0].titulo')

# Selecionando todos os títulos de livros
run_jmespath_query('livraria.livros[*].titulo')

# Selecionando o nome do segundo usuário
run_jmespath_query('usuarios[1].nome')

# Selecionando chave com traço (necessita aspas invertidas ou de ponto)
run_jmespath_query('configuracoes."chave-com-traco"')
run_jmespath_query('configuracoes.`chave-com-traco`') # Aspas invertidas são preferenciais para chaves complexas


--- JMESPath Query: 'livraria.titulo' ---
  Resultado: Catálogo de Livros Técnicos

--- JMESPath Query: 'livraria.livros[0].titulo' ---
  Resultado: Aprendendo JSONPath

--- JMESPath Query: 'livraria.livros[*].titulo' ---
  Resultado: ['Aprendendo JSONPath', 'Dominando JMESPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']

--- JMESPath Query: 'usuarios[1].nome' ---
  Resultado: Bruno

--- JMESPath Query: 'configuracoes."chave-com-traco"' ---
  Nenhum resultado encontrado ou resultado vazio.

--- JMESPath Query: 'configuracoes.`chave-com-traco`' ---
  Erro ao executar query: Expecting: ['quoted_identifier', 'unquoted_identifier', 'lbracket', 'lbrace'], got: literal: Parse error at column 14, token "chave-com-traco" (LITERAL), for expression:
"configuracoes.`chave-com-traco`"
               ^


### 2. Sintaxe e Recursos JMESPath

JMESPath tem uma sintaxe rica e poderosa para diversas operações:

  - **`chave`**: Acesso a membros de objetos. Ex: `livraria.titulo`
  - **`[índice]`**: Acesso a elementos de arrays. Ex: `livros[0]`
  - **`[*]`**: Wildcard para todos os elementos em um array ou todos os valores em um objeto. Ex: `livros[*].titulo`
  - **`[]` (Projeções)**: Permite transformar a estrutura dos dados.
      - **`[?condição]` (Filtro)**: Filtra elementos de um array com base em uma condição. Ex: `livros[?disponivel]`
      - **`{chave: expressão, ...}` (Objeto Projection)**: Cria um novo objeto (dicionário) com chaves e valores derivados. Ex: `livros[*].{Titulo: titulo, Preco: preco}`
      - **`[expressão]` (Array Projection)**: Aplica uma expressão a cada elemento de um array e coleta os resultados em um novo array. Ex: `livros[*].autores[0]` (primeiro autor de cada livro)
  - **`|` (Pipe)**: Encadeia operações, passando o resultado da expressão anterior para a próxima. Ex: `livros[*].autores | [0]` (pega todos os autores e então o primeiro de cada lista)
  - **Funções:** JMESPath tem uma rica biblioteca de funções built-in. As funções são chamadas com `funcao(argumento1, argumento2, ...)`.
      - **Tipos de literais:**
          - **Strings:** `'meu texto'`, `"meu texto"`.
          - **Números:** `123`, `12.3`.
          - **Booleanos:** `true`, `false`.
          - **Nulos:** `null`.
          - **Backticks (`` ` ``):** Usados para literais que podem confundir o parser, como números em comparações com strings ou chaves de objeto. Ex: ` preco >  `25.00\`\`
  - **`@`**: Representa o elemento atual no contexto de um filtro ou projeção.


In [5]:
print("\n--- Filtros JMESPath ---")

# Filtrando livros disponíveis (mais conciso que JSONPath)
run_jmespath_query('livraria.livros[?disponivel].titulo')

# Filtrando livros não disponíveis
run_jmespath_query('livraria.livros[?!disponivel].titulo')

# Filtrando livros com preço maior que 25 E da Editora Inovação
run_jmespath_query('livraria.livros[?preco > `25.00` && editora.nome == `Editora Inovação`].titulo')

# Filtrando usuários administradores
run_jmespath_query('usuarios[?isAdmin].nome')

print("\n--- Projeções JMESPath (Remodelagem de Dados) ---")

# Remodelando para pegar título e primeiro autor de cada livro
run_jmespath_query('livraria.livros[*].{Titulo: titulo, PrimeiroAutor: autores[0]}')

# Remodelando para pegar título e cidade da editora para livros com preço menor que 30
run_jmespath_query('livraria.livros[?preco < `30`].{Titulo: titulo, CidadeEditora: editora.cidade}')

print("\n--- Funções JMESPath ---")

# length(): Comprimento de arrays ou strings
run_jmespath_query('livraria.livros[0].autores | length(@)') # Comprimento do array de autores do primeiro livro
run_jmespath_query('livraria.titulo | length(@)') # Comprimento da string do título

# contains(): Verifica se um array contém um valor ou se uma string contém um substring
run_jmespath_query('livraria.livros[?contains(tags, `Programação`)].titulo') # Livros com a tag 'Programação'
run_jmespath_query('livraria.livros[?contains(titulo, `JSONPath`)].titulo') # Livros cujo título contém 'JSONPath'

# join(): Junta elementos de um array de strings com um delimitador
run_jmespath_query('livraria.livros[0].autores | join(`, `, @)') # Juntando autores do primeiro livro

# reverse(): Inverte um array ou string
run_jmespath_query('usuarios[0].ultimosAcessos | reverse(@)') # Invertendo a ordem dos acessos

# max()/min(): Pega o valor máximo/mínimo de um array de números
run_jmespath_query('livraria.livros[*].preco | max(@)') # Preço máximo entre os livros

# sum(): Soma os elementos de um array de números
run_jmespath_query('livraria.livros[*].preco | sum(@)') # Soma dos preços dos livros

# type(): Retorna o tipo do valor
run_jmespath_query('livraria.livros[0].preco | type(@)')

# values(): Retorna os valores de um objeto como um array
run_jmespath_query('livraria.editora | values(@)') # Isso não funcionaria aqui, pois editora não é um objeto raiz de busca.
# Exemplo correto de values:
run_jmespath_query('livraria.{dados: editora | values(@)}') # Usa projeção para pegar os valores da editora

# keys(): Retorna as chaves de um objeto como um array
run_jmespath_query('livraria.editora | keys(@)')

# Sort_by(): Ordena um array de objetos por uma chave
run_jmespath_query('livraria.livros | sort_by(@, &preco).titulo') # Ordena livros por preço e pega os títulos

print("\n--- Operador Pipe para Transformações em Sequência ---")
# Seleciona os livros disponíveis, depois para cada um, pega o título e o primeiro autor,
# e então ordena o resultado final pelo título
run_jmespath_query('livraria.livros[?disponivel].{Titulo: titulo, PrimeiroAutor: autores[0]} | sort_by(@, &Titulo)')

print("\n--- JMESPath: Fatiamento de Array ---")
# JMESPath não tem a mesma sintaxe de slicing [start:end:step] do JSONPath.
# Para fatiar arrays, você geralmente combinaria filtros ou funções se necessário,
# ou usaria a indexação direta. Por exemplo, para os dois primeiros:
run_jmespath_query('livraria.livros[0:2]') # JMESPath apenas aceita um único índice ou [*]. Slicing como [0:2] não é padrão aqui.
# Para os dois primeiros livros em JMESPath, você geralmente faria:
run_jmespath_query('livraria.livros[0], livraria.livros[1]') # Não é ideal, mas ilustra a limitação.
# Ou se for um subconjunto simples, pode-se usar a projeção de array com indices
# livros[0:2] NÃO É UMA SINTAXE PADRÃO DE JMESPATH PARA SLICING.
# Em JMESPath, fatiamento é mais complexo, geralmente feito com funções ou na linguagem hospedeira.
# Para o primeiro e segundo livro, você pegaria os elementos individualmente ou filtraria.
# Ex: Para obter o primeiro e o segundo livro, você faria:
run_jmespath_query('[livraria.livros[0], livraria.livros[1]]')


--- Filtros JMESPath ---

--- JMESPath Query: 'livraria.livros[?disponivel].titulo' ---
  Resultado: ['Aprendendo JSONPath', 'Introdução ao Web Scraping', 'Python para Iniciantes']

--- JMESPath Query: 'livraria.livros[?!disponivel].titulo' ---
  Resultado: ['Dominando JMESPath']

--- JMESPath Query: 'livraria.livros[?preco > `25.00` && editora.nome == `Editora Inovação`].titulo' ---
  Resultado: ['Aprendendo JSONPath']

--- JMESPath Query: 'usuarios[?isAdmin].nome' ---
  Resultado: ['Ana']

--- Projeções JMESPath (Remodelagem de Dados) ---

--- JMESPath Query: 'livraria.livros[*].{Titulo: titulo, PrimeiroAutor: autores[0]}' ---
  Resultado: [{'Titulo': 'Aprendendo JSONPath', 'PrimeiroAutor': 'Alice'}, {'Titulo': 'Dominando JMESPath', 'PrimeiroAutor': 'Charlie'}, {'Titulo': 'Introdução ao Web Scraping', 'PrimeiroAutor': 'David'}, {'Titulo': 'Python para Iniciantes', 'PrimeiroAutor': 'Grace'}]

--- JMESPath Query: 'livraria.livros[?preco < `30`].{Titulo: titulo, CidadeEditora: editora.

### 3. Compilação de Expressões (`jmespath.compile()`)

Embora `jmespath.search()` seja conveniente para uso único, para expressões que serão usadas repetidamente, é mais eficiente compilá-las uma vez:

  - **`jmespath.compile(expression)`**:
      - **O que faz:** Compila a expressão JMESPath em um objeto de expressão.
      - **Retorna:** Um objeto `jmespath.parser.ParsedResult` que possui um método `search()` próprio.


In [6]:
# Compila a expressão uma vez
compiled_expression = jmespath.compile('livraria.livros[?preco > `30`].titulo')

# Usa a expressão compilada múltiplas vezes (aqui, apenas uma vez para demonstração)
result_compiled = compiled_expression.search(json_data)
print(f"Resultado da expressão compilada: {result_compiled}")

# Você pode usar a mesma expressão compilada com outros dados
other_data = {"livraria": {"livros": [{"titulo": "Novo Livro", "preco": 40.0}]}}
result_other_data = compiled_expression.search(other_data)
print(f"Resultado com outros dados: {result_other_data}")

Resultado da expressão compilada: ['Python para Iniciantes']
Resultado com outros dados: ['Novo Livro']


## Exceções Comuns

  - **`jmespath.exceptions.JMESPathError`**: A classe base para todos os erros JMESPath.
  - **`jmespath.exceptions.LexerError`**: Erro de tokenização na expressão JMESPath (caracteres inesperados).
  - **`jmespath.exceptions.ParseError`**: Erro de sintaxe na expressão JMESPath.
  - **`jmespath.exceptions.ArityError`**: Função chamada com o número incorreto de argumentos.
  - **`jmespath.exceptions.JMESPathTypeError`**: Tipo de dado inválido fornecido para uma operação ou função.
  - **`jmespath.exceptions.UnknownFunctionError`**: Tentativa de chamar uma função que não existe.
